In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [4]:
df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=['Tanggal']) #parse = untuk ngubah kolom Tanggal jadi datetime
df = df.sort_values('Tanggal')
df = df.reset_index(drop=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   object        
 2   Temperatur Maksimum                 4020 non-null   object        
 3   Temperatur Rata-rata                4020 non-null   object        
 4   Kelembapan Rata-rata                4020 non-null   object        
 5   Curah Hujan (mm)                    4020 non-null   object        
 6   Lamanya Penyinaran Matahari         4020 non-null   object        
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [6]:
checkstripvalue = (df['Curah Hujan (mm)'] == "-").sum()
print(checkstripvalue)

497


In [8]:
df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)

df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].astype(float)

In [9]:
checkstripvalue2 = (df['Curah Hujan (mm)'] == 8888).sum()
print(checkstripvalue2)

209


In [64]:
# mapping = {
#     'C': 0, #Calm
#     'N': 0, #North
#     'NE': 45, #North East
#     'E': 90, #East
#     'SE': 135, #South East 
#     'S': 180, #South
#     'SW': 225, #South West 
#     'W': 270, #West 
#     'NW': 315 #North West
# }

# df['Arah Angin Terbanyak (°)'] = df['Arah Angin Terbanyak (°)'].map(mapping)

# Fill Missing Values using RF Imputer

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

df2 = pd.read_excel(
    "Updated_Data_Historis_2015_2025.xlsx",
    parse_dates=["Tanggal"]
)
df2 = df2.sort_values("Tanggal").reset_index(drop=True)

invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ", "8888", 8888]
df2 = df2.replace(invalid_values, np.nan)

# Tangkap SEMUA blank aneh dari Excel
df2 = df2.replace(r'^\s*$', np.nan, regex=True)

col = "Arah Angin Terbanyak (°)"

df2[col] = (
    df2[col].astype(str)
            .str.strip()
            .str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False)
)

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}

df2[col] = df2[col].map(mapping)

for c in df2.columns:
    if c != "Tanggal":
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

target = "Curah Hujan (mm)"

feature_cols = [
    c for c in df2.columns
    if c not in ["Tanggal", target]
]
df2[feature_cols] = df2[feature_cols].fillna(
    df2[feature_cols].median()
)

train_df = df2[df2[target].notna()]
pred_df  = df2[df2[target].isna()]

X_train = train_df[feature_cols]
y_train = train_df[target]

X_pred = pred_df[feature_cols]

rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

df2.loc[df2[target].isna(), target] = rf.predict(X_pred)

df2.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
print("File disimpan: Ready_To_Use_Updated_RF.xlsx")


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\4074261390.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2 = df2.replace(invalid_values, np.nan)


File disimpan: Ready_To_Use_Updated_RF.xlsx


In [11]:
data1 = pd.read_excel('Ready_To_Use_Updated_RF.xlsx')

In [12]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   float64       
 2   Temperatur Maksimum                 4020 non-null   float64       
 3   Temperatur Rata-rata                4020 non-null   float64       
 4   Kelembapan Rata-rata                4020 non-null   int64         
 5   Curah Hujan (mm)                    4020 non-null   float64       
 6   Lamanya Penyinaran Matahari         4020 non-null   float64       
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [13]:
data1.isna().sum().sort_values(ascending=False)


Tanggal                               0
Temperatur Minimum                    0
Temperatur Maksimum                   0
Temperatur Rata-rata                  0
Kelembapan Rata-rata                  0
Curah Hujan (mm)                      0
Lamanya Penyinaran Matahari           0
Kecepatan Angin Maksimum              0
Arah Angin Saat Kecepatan Maksimum    0
Kecepatan Angin Rata-rata             0
Arah Angin Terbanyak (°)              0
dtype: int64

# Feature Engineering

In [ ]:
# === 1. Load Data (setelah imputasi RF yang tadi) ===

df = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")

# Pastikan Tanggal datetime
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

# Sort berdasarkan tanggal
df = df.sort_values('Tanggal')

# === 2. Tentukan kolom fitur yang akan dibuatkan lag ===
# Semua kolom kecuali Tanggal
feature_cols = df.columns.drop('Tanggal')

# === 3. Buat fungsi untuk generate lag otomatis ===
def create_lag_features(df, cols, max_lag=30):
    df = df.copy()

    for col in cols:
        for lag in range(1, max_lag + 1):
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)

    return df

# === 4. Buat lag features t-1 sampai t-30 ===
df_lagged = create_lag_features(df, feature_cols, max_lag=30)

# === 5. Drop baris awal yang menjadi NaN akibat lag ===
df_lagged = df_lagged.dropna().reset_index(drop=True)

# === 6. Simpan hasilnya ===
df_lagged.to_excel("df_feature_engineered.xlsx", index=False)
print("Shape:", df_lagged.shape)

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\2570611351.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\2570611351.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\2570611351.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

Shape: (3990, 311)


In [16]:

df3 = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")

# Pastikan datetime & sort
df3['Tanggal'] = pd.to_datetime(df3['Tanggal'])
df3 = df3.sort_values('Tanggal').reset_index(drop=True)

feature_cols = df3.columns.drop('Tanggal')

def create_lag_features(df3, cols, max_lag=30):
    df3_lag = df3.copy()

    for col in cols:
        for lag in range(1, max_lag + 1):
            df3_lag[f"{col}_lag_{lag}"] = df3_lag[col].shift(lag)

    return df3_lag

df3_lagged = create_lag_features(
    df3,
    feature_cols,
    max_lag=30
)

df3_lagged = df3_lagged.iloc[30:].reset_index(drop=True)

df3_lagged.to_excel("df3_feature_engineered.xlsx", index=False)

print("✅ Lag features t-1 s/d t-30 berhasil dibuat")
print("Shape:", df3_lagged.shape)


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\125165152.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3_lag[f"{col}_lag_{lag}"] = df3_lag[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\125165152.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3_lag[f"{col}_lag_{lag}"] = df3_lag[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_2896\125165152.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

✅ Lag features t-1 s/d t-30 berhasil dibuat
Shape: (3990, 311)


In [17]:
data = pd.read_excel('df3_feature_engineered.xlsx')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3990 entries, 0 to 3989
Columns: 311 entries, Tanggal to Arah Angin Terbanyak (°)_lag_30
dtypes: datetime64[ns](1), float64(155), int64(155)
memory usage: 9.5 MB


# Data Modelling 

In [24]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor


df4 = pd.read_excel("df3_feature_engineered.xlsx")

# Target klasifikasi
df4["Rain_Event"] = (df4["Curah Hujan (mm)"] > 0).astype(int)

# Fitur untuk model
feature_cols = df4.columns.drop([
    "Tanggal",
    "Curah Hujan (mm)",
    "Rain_Event"
])

X = df4[feature_cols]
y_cls = df4["Rain_Event"]

# Time Series Split
tscv = TimeSeriesSplit(n_splits=5)
train_idx, test_idx = list(tscv.split(X))[-1]

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train_cls, y_test_cls = y_cls.iloc[train_idx], y_cls.iloc[test_idx]

# Model klasifikasi
clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

clf.fit(X_train, y_train_cls)

# Evaluasi
y_pred_prob = clf.predict_proba(X_test)[:, 1]
print("Stage 1 — Rain Event Accuracy:", accuracy_score(y_test_cls, y_pred_cls))

# Hanya data hujan (>0)
df4_rain = df4[df4["Curah Hujan (mm)"] > 0].copy()

X_rain = df4_rain[feature_cols]
y_rain = np.log1p(df4_rain["Curah Hujan (mm)"])

train_idx_r, test_idx_r = list(tscv.split(X_rain))[-1]

X_train_r, X_test_r = X_rain.iloc[train_idx_r], X_rain.iloc[test_idx_r]
y_train_r, y_test_r = y_rain.iloc[train_idx_r], y_rain.iloc[test_idx_r]

# Model regresi
reg = XGBRegressor(
    n_estimators=800,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weights=5,
    reg_alpha=0.5,
    reg_lambda=1.5,
    objective="reg:squarederror",
    random_state=42
)

reg.fit(X_train_r, y_train_r)

# Evaluasi regresi
y_pred_reg = np.expm1(reg.predict(X_test_r))
print("Stage 2 — Rain Amount MAE:", mean_absolute_error(y_test_r, y_pred_reg))

final_pred = []
rain_threshold = 0.3
for i in range(len(X_test)):
    rain_prob = y_pred_prob[i]

    if rain_prob < rain_threshold:
        final_pred.append(0)
    else:
        rain_amount = np.expm1(
            reg.predict(X_test.iloc[[i]])[0]
        )
        final_pred.append(rain_prob * rain_amount)

final_pred = np.array(final_pred)

# Evaluasi akhir
y_true = df4.iloc[test_idx]["Curah Hujan (mm)"]

print("FINAL MAE (Two-Stage):", mean_absolute_error(y_true, final_pred))


result = df4.iloc[test_idx][["Tanggal"]].copy()
result["Actual_Rain_mm"] = y_true.values
result["Predicted_Rain_mm"] = final_pred

result.to_excel("two_stage_prediction_result.xlsx", index=False)

print("Selesai! Hasil tersimpan di two_stage_prediction_result.xlsx")


Stage 1 — Rain Event Accuracy: 0.7413533834586467


c:\Users\Lenovo E15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [15:16:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "min_child_weights" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Stage 2 — Rain Amount MAE: 4.894442009342379
FINAL MAE (Two-Stage): 5.647151553043395
Selesai! Hasil tersimpan di two_stage_prediction_result.xlsx


In [78]:
print("NaN di X_test:", np.isnan(X_test).sum())
print("NaN di y_test:", np.isnan(y_test).sum())
print("NaN di y_pred:", np.isnan(y_pred).sum())

NaN di X_test: 0
NaN di y_test: 0
NaN di y_pred: 792
